In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import numpy as np
import matplotlib.pyplot as plt                        
import torch
import pandas as pd
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Training on A1 BenchMark data

In [3]:
# load one dataset

ts_data = pd.read_csv('./ydata-labeled-time-series-anomalies-v1_0/A1Benchmark/real_60.csv',index_col = 0)
#ts_data = ts_data.astype('float')
ts_data.head()

,value,is_anomaly
timestamp,,
1,1.265278,0
2,1.100833,0
3,1.147778,0
4,1.053889,0
5,1.051944,0


In [4]:
ts_data[['is_anomaly','value']].groupby('is_anomaly').count()

,value
is_anomaly,
0,1445
1,16


In [5]:
ts_data.shape

(1461, 2)

In [6]:
# define train, test and validation datasets

train_percent = int(0.3*len(ts_data))
valid_percent = int(0.1*len(ts_data))
test_percent = int(0.6*len(ts_data))

train_data = list(ts_data.iloc[:train_percent,0])
valid_data = list(ts_data.iloc[train_percent:train_percent+valid_percent,0])
test_data = list(ts_data.iloc[train_percent+valid_percent:,0])

In [7]:
# define parameters

w = 45
pred_window = 1
filter1_size = 128
filter2_size = 32
kernel_size = 2
stride = 1
pool_size = 2

In [8]:
# generate subsequences on which we will train

def get_subsequences(data):
    X = []
    Y = []
    
    for i in range(len(data) - w -pred_window):
        X.append(data[i:i+w])
        Y.append(data[i+w:i+w+pred_window])
    return np.array(X),np.array(Y)

trainX,trainY = get_subsequences(train_data)
trainX = np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))

validX,validY = get_subsequences(valid_data)
validX = np.reshape(validX,(validX.shape[0],1,validX.shape[1]))

testX,testY = get_subsequences(test_data)
testX = np.reshape(testX,(testX.shape[0],1,testX.shape[1]))


In [9]:
#  CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## layers of a CNN
        
        self.conv1 = nn.Conv1d(1,filter1_size,kernel_size,stride,padding = 0)
        
        self.conv2 = nn.Conv1d(filter1_size,filter2_size,kernel_size,stride,padding = 0)

        self.maxpool = nn.MaxPool1d(pool_size)
        
        self.dim1 = int(0.5*(0.5*(w-1)-1)) * filter2_size
        
        self.lin1 = nn.Linear(self.dim1,pred_window )
        #self.lin2 = nn.Linear(1000,pred_window)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        
        #convolution layer 1
        x = (F.relu(self.conv1(x)))
        x = self.maxpool(x)
        #print(x.shape)
        #x = self.dropout(x)

        #convolution layer 2
        x = (F.relu(self.conv2(x)))
        x = self.maxpool(x)
        #x = self.dropout(x)

        #print(x.shape)


        #print(x.shape)
        #print(int(0.25* (w) * filter2_size))
        x = x.view(-1,self.dim1)
        
        x = self.dropout(x)
        x = self.lin1(x)
        #x = self.dropout(x)
        #x = self.lin2(x)
        
        return x

In [10]:
# define CNN model

model_A1 = Net()
print(model_A1)

Net(
  (conv1): Conv1d(1, 128, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(128, 32, kernel_size=(2,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (lin1): Linear(in_features=320, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [11]:
# define optimizer

criterion_scratch = nn.L1Loss()
optimizer_scratch = optim.Adam(model_A1.parameters(), lr = 1e-5,weight_decay=1e-6)

In [12]:
# function for training the model (also checks on validation data)

def train_valid(n_epochs, trainX,trainY, validX,validY,model, optimizer, criterion,save_path,freq = 5):
    """returns trained model"""

    target_train = torch.tensor(trainY).type('torch.FloatTensor')
    data_train = torch.tensor(trainX).type('torch.FloatTensor')
    
    target_valid = torch.tensor(validY).type('torch.FloatTensor')
    data_valid = torch.tensor(validX).type('torch.FloatTensor')
    
    train_loss_min = np.Inf
    valid_loss_min = np.Inf
    last_valid_loss= 0
    
    for epoch in range(1, n_epochs+1):
        
        ###################
        # training the model #
        ###################
        model.train()

        #print(data.shape)


        optimizer.zero_grad()
        output = model(data_train)
        loss = criterion(output, target_train)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()
        
        ###################
        # Validation #
        ###################
        model.eval()
        output_valid = model(data_valid)
        
        loss_valid = criterion(output_valid, target_valid)
        valid_loss = loss_valid.item()
        if(valid_loss == last_valid_loss):
            print('problem')
            
        last_valid_loss = valid_loss
        if(epoch%freq == 0):
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss
                ))
            
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    return model,output

In [13]:
# train model

model_A1,out = train_valid(500, trainX,trainY,validX,validY, model_A1, optimizer_scratch, 
                      criterion_scratch, 'model_A1.pt',freq = 10)

Validation loss decreased (inf --> 0.950132).  Saving model ...
Validation loss decreased (0.950132 --> 0.948436).  Saving model ...
Validation loss decreased (0.948436 --> 0.946736).  Saving model ...
Validation loss decreased (0.946736 --> 0.945034).  Saving model ...
Validation loss decreased (0.945034 --> 0.943330).  Saving model ...
Validation loss decreased (0.943330 --> 0.941626).  Saving model ...
Validation loss decreased (0.941626 --> 0.939920).  Saving model ...
Validation loss decreased (0.939920 --> 0.938214).  Saving model ...
Validation loss decreased (0.938214 --> 0.936505).  Saving model ...
Epoch: 10 	Training Loss: 0.949167 	Validation Loss: 0.934794
Validation loss decreased (0.936505 --> 0.934794).  Saving model ...
Validation loss decreased (0.934794 --> 0.933081).  Saving model ...
Validation loss decreased (0.933081 --> 0.931362).  Saving model ...
Validation loss decreased (0.931362 --> 0.929636).  Saving model ...
Validation loss decreased (0.929636 --> 0.9279

Validation loss decreased (0.744456 --> 0.742392).  Saving model ...
Validation loss decreased (0.742392 --> 0.740325).  Saving model ...
Validation loss decreased (0.740325 --> 0.738253).  Saving model ...
Validation loss decreased (0.738253 --> 0.736179).  Saving model ...
Validation loss decreased (0.736179 --> 0.734101).  Saving model ...
Validation loss decreased (0.734101 --> 0.732022).  Saving model ...
Validation loss decreased (0.732022 --> 0.729940).  Saving model ...
Validation loss decreased (0.729940 --> 0.727856).  Saving model ...
Validation loss decreased (0.727856 --> 0.725773).  Saving model ...
Epoch: 120 	Training Loss: 0.746423 	Validation Loss: 0.723686
Validation loss decreased (0.725773 --> 0.723686).  Saving model ...
Validation loss decreased (0.723686 --> 0.721597).  Saving model ...
Validation loss decreased (0.721597 --> 0.719506).  Saving model ...
Validation loss decreased (0.719506 --> 0.717414).  Saving model ...
Validation loss decreased (0.717414 --> 

Validation loss decreased (0.505292 --> 0.502944).  Saving model ...
Validation loss decreased (0.502944 --> 0.500589).  Saving model ...
Validation loss decreased (0.500589 --> 0.498228).  Saving model ...
Validation loss decreased (0.498228 --> 0.495863).  Saving model ...
Validation loss decreased (0.495863 --> 0.493496).  Saving model ...
Validation loss decreased (0.493496 --> 0.491122).  Saving model ...
Validation loss decreased (0.491122 --> 0.488744).  Saving model ...
Validation loss decreased (0.488744 --> 0.486356).  Saving model ...
Validation loss decreased (0.486356 --> 0.483965).  Saving model ...
Epoch: 230 	Training Loss: 0.501860 	Validation Loss: 0.481568
Validation loss decreased (0.483965 --> 0.481568).  Saving model ...
Validation loss decreased (0.481568 --> 0.479165).  Saving model ...
Validation loss decreased (0.479165 --> 0.476757).  Saving model ...
Validation loss decreased (0.476757 --> 0.474343).  Saving model ...
Validation loss decreased (0.474343 --> 

Validation loss decreased (0.234213 --> 0.232053).  Saving model ...
Validation loss decreased (0.232053 --> 0.229929).  Saving model ...
Validation loss decreased (0.229929 --> 0.227866).  Saving model ...
Validation loss decreased (0.227866 --> 0.225812).  Saving model ...
Validation loss decreased (0.225812 --> 0.223775).  Saving model ...
Validation loss decreased (0.223775 --> 0.221740).  Saving model ...
Validation loss decreased (0.221740 --> 0.219706).  Saving model ...
Validation loss decreased (0.219706 --> 0.217699).  Saving model ...
Validation loss decreased (0.217699 --> 0.215743).  Saving model ...
Epoch: 340 	Training Loss: 0.260589 	Validation Loss: 0.213817
Validation loss decreased (0.215743 --> 0.213817).  Saving model ...
Validation loss decreased (0.213817 --> 0.211924).  Saving model ...
Validation loss decreased (0.211924 --> 0.210113).  Saving model ...
Validation loss decreased (0.210113 --> 0.208317).  Saving model ...
Validation loss decreased (0.208317 --> 

Validation loss decreased (0.147313 --> 0.147260).  Saving model ...
Validation loss decreased (0.147260 --> 0.147209).  Saving model ...
Validation loss decreased (0.147209 --> 0.147158).  Saving model ...
Validation loss decreased (0.147158 --> 0.147108).  Saving model ...
Validation loss decreased (0.147108 --> 0.147061).  Saving model ...
Validation loss decreased (0.147061 --> 0.147014).  Saving model ...
Validation loss decreased (0.147014 --> 0.146968).  Saving model ...
Epoch: 450 	Training Loss: 0.173749 	Validation Loss: 0.146923
Validation loss decreased (0.146968 --> 0.146923).  Saving model ...
Validation loss decreased (0.146923 --> 0.146879).  Saving model ...
Validation loss decreased (0.146879 --> 0.146834).  Saving model ...
Validation loss decreased (0.146834 --> 0.146791).  Saving model ...
Validation loss decreased (0.146791 --> 0.146747).  Saving model ...
Validation loss decreased (0.146747 --> 0.146704).  Saving model ...
Validation loss decreased (0.146704 --> 

In [14]:
# load best model (saved previously during training)

model_A1.load_state_dict(torch.load('model_A1.pt'))

<All keys matched successfully>

In [15]:
# make predictions

test_tensor =  torch.tensor(testX).type('torch.FloatTensor')
model_A1.eval()
out = model_A1(test_tensor)
out = out.detach().numpy()

In [16]:
df_out = pd.DataFrame()
df_out['pred'] = out[:,0]
df_out['actual'] = testY[:,0]
#df_out.index = ts_data.index[train_percent + valid_percent:len(ts_data)-w-pred_window]

df_out.tail()

,pred,actual
826,0.941016,2.354444
827,0.951111,2.680556
828,0.954491,3.063889
829,0.967551,2.462778
830,0.971499,1.616667


In [17]:
# compute error (actual - pred)

df_out['error'] = np.abs(df_out['pred'] - df_out['actual'])
df_out['error_n'] = (df_out['error'] - df_out['error'].mean())/df_out['error'].std()
df_out.index = ts_data.index[train_percent + valid_percent +w+pred_window-1:-1]

In [18]:
# check whether error is more than the threshold

thresh = df_out.loc[df_out['error_n'].abs() > 3]
thresh

,pred,actual,error,error_n
timestamp,,,,
844,0.871605,3.982222,3.110617,6.303662
1208,1.055828,5.646111,4.590283,9.530605
1209,1.051940,10.452778,9.400838,20.021752
1210,1.020918,4.237778,3.216860,6.535363
1211,1.087213,3.192222,2.105010,4.110573
1457,0.951111,2.680556,1.729444,3.291517
1458,0.954491,3.063889,2.109398,4.120144


In [19]:
# calc TP, FN, FP, TN

positives = ts_data.loc[df_out.index].loc[ts_data.is_anomaly == 1].index
negatives = ts_data.loc[df_out.index].loc[ts_data.is_anomaly == 0].index
tp = []
fn = []
fp = []
tn = []
for p in positives:
    if p in thresh.index:
        tp.append(p)
    else:
        fn.append(p)

for n in negatives:
    if n in thresh.index:
        fp.append(n)
    else:
        tn.append(n)

In [20]:
# calc F-score

recall = len(tp)/(len(tp)+len(fn))
precision = len(tp)/(len(tp)+len(fp))
F_score = 2* recall*precision/(recall + precision)
F_score

0.7777777777777778